In [1]:
#!/usr/bin/python

import sys
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.cluster import KMeans

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

sys.path.append("../tools/")
# from feature_format import featureFormat, targetFeatureSplit
from dos2unix import crlf_to_lf
# from tester import dump_classifier_and_data

In [2]:
### Load data set.
X_train = None
y_train = None
X_test = None
y_test = None
with open(file='X_train.pkl', mode='rb') as file:
    X_train = pickle.load(file=file)
with open(file='y_train.pkl', mode='rb') as file:
    y_train = pickle.load(file=file)
with open(file='X_test.pkl', mode='rb') as file:
    X_test = pickle.load(file=file)
with open(file='y_test.pkl', mode='rb') as file:
    y_test = pickle.load(file=file)

fin_features = None
pay_features = None
stock_features = None
email_features = None
features_list = None
with open('fin_features.pkl', 'rb') as file:
    fin_features = pickle.load(file=file)
with open('pay_features.pkl', 'rb') as file:
    pay_features = pickle.load(file=file)
with open('stock_features.pkl', 'rb') as file:
    stock_features = pickle.load(file=file)
with open('email_features.pkl', 'rb') as file:
    email_features = pickle.load(file=file)
with open('features_list.pkl', 'rb') as file:
    features_list = pickle.load(file=file)

In [3]:
print('X_train')
X_train.head()
X_train.info()
print('y_train')
y_train.head()
y_train.dtypes
print('X_test')
X_test.head()
X_test.info()
print('y_test')
y_test.head()
y_test.dtypes

X_train


,salary,bonus,long_term_incentive,deferred_income,deferral_payments,other,expenses,total_payments,exercised_stock_options,restricted_stock,total_stock_value,to_messages,from_poi_to_this_person,from_messages,from_this_person_to_poi,shared_receipt_with_poi
ECHOLS JOHN B,182245.0,200000.0,1920000.0,NaN,NaN,53775.0,21530.0,2692324.0,601438.0,407503.0,1008941.0,NaN,NaN,NaN,NaN,NaN
KAMINSKI WINCENTY J,275101.0,400000.0,323466.0,NaN,NaN,4669.0,83585.0,1086821.0,850010.0,126027.0,976037.0,4607.0,41.0,8000.0,171.0,583.0
BOWEN JR RAYMOND M,278601.0,1350000.0,974293.0,-5104.0,NaN,1621.0,65907.0,2669589.0,NaN,252055.0,252055.0,1858.0,140.0,27.0,15.0,1593.0
PIPER GREGORY F,197091.0,400000.0,NaN,-33333.0,1130036.0,778.0,43057.0,1737629.0,880290.0,409554.0,880290.0,1238.0,61.0,222.0,48.0,742.0
HAYES ROBERT E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,151418.0,151418.0,504.0,16.0,12.0,0.0,50.0


<class 'pandas.core.frame.DataFrame'>
Index: 86 entries, ECHOLS JOHN B to MURRAY JULIA H
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   salary                   62 non-null     float64
 1   bonus                    54 non-null     float64
 2   long_term_incentive      40 non-null     float64
 3   deferred_income          22 non-null     float64
 4   deferral_payments        23 non-null     float64
 5   other                    59 non-null     float64
 6   expenses                 62 non-null     float64
 7   total_payments           72 non-null     float64
 8   exercised_stock_options  64 non-null     float64
 9   restricted_stock         72 non-null     float64
 10  total_stock_value        81 non-null     float64
 11  to_messages              60 non-null     float64
 12  from_poi_to_this_person  60 non-null     float64
 13  from_messages            60 non-null     float64
 14  from_this

ECHOLS JOHN B          0
KAMINSKI WINCENTY J    0
BOWEN JR RAYMOND M     1
PIPER GREGORY F        0
HAYES ROBERT E         0
Name: poi, dtype: int32

dtype('int32')

X_test


,salary,bonus,long_term_incentive,deferred_income,deferral_payments,other,expenses,total_payments,exercised_stock_options,restricted_stock,total_stock_value,to_messages,from_poi_to_this_person,from_messages,from_this_person_to_poi,shared_receipt_with_poi
WESTFAHL RICHARD K,63744.0,NaN,256191.0,-10800.0,NaN,401130.0,51870.0,762135.0,NaN,384930.0,384930.0,NaN,NaN,NaN,NaN,NaN
SHELBY REX,211844.0,200000.0,NaN,-4167.0,NaN,1573324.0,22884.0,2003885.0,1624396.0,869220.0,2493616.0,225.0,13.0,39.0,14.0,91.0
DEFFNER JOSEPH M,206121.0,600000.0,335349.0,NaN,NaN,25553.0,41626.0,1208649.0,17378.0,141833.0,159211.0,714.0,115.0,74.0,4.0,552.0
WASAFF GEORGE,259996.0,325000.0,200000.0,-583325.0,831299.0,1425.0,NaN,1034395.0,1668260.0,388167.0,2056427.0,400.0,22.0,30.0,7.0,337.0
HANNON KEVIN P,243293.0,1500000.0,1617011.0,-3117011.0,NaN,11350.0,34039.0,288682.0,5538001.0,853064.0,6391065.0,1045.0,32.0,32.0,21.0,1035.0


<class 'pandas.core.frame.DataFrame'>
Index: 37 entries, WESTFAHL RICHARD K to PRENTICE JAMES
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   salary                   31 non-null     float64
 1   bonus                    27 non-null     float64
 2   long_term_incentive      25 non-null     float64
 3   deferred_income          18 non-null     float64
 4   deferral_payments        12 non-null     float64
 5   other                    29 non-null     float64
 6   expenses                 24 non-null     float64
 7   total_payments           37 non-null     float64
 8   exercised_stock_options  27 non-null     float64
 9   restricted_stock         32 non-null     float64
 10  total_stock_value        36 non-null     float64
 11  to_messages              26 non-null     float64
 12  from_poi_to_this_person  26 non-null     float64
 13  from_messages            26 non-null     float64
 14  from

WESTFAHL RICHARD K    0
SHELBY REX            1
DEFFNER JOSEPH M      0
WASAFF GEORGE         0
HANNON KEVIN P        1
Name: poi, dtype: int32

dtype('int32')

In [4]:
## Drop 'other' because it's ill-defined and seems overly represented within
## important features. The nebulous nature of it seems like a good fit for
## fraud, but high gross 'other' amounts are more correlated with nonPOIs than
## POIs if anything.
X_train.drop(columns=['other'], inplace=True)
X_test.drop(columns=['other'], inplace=True)
fin_features = [feat for feat in fin_features if feat not in ['other']]
pay_features = [feat for feat in pay_features if feat not in ['other']]
features_list = [feat for feat in fin_features if feat not in ['other']]

### Task 3: Create new feature(s)

#### Feature ideas:

from_poi_to_this_person / to_messages
shared_receipt_with_poi / to_messages
from_this_person_to_poi / from_messages
from_poi_to_this_person / shared_receipt_with_poi

pay features / pay totals
stock features / stock totals

Label all as outliers or not before removing outliers, allowing feature significance and PCA to decide what's important. May need to more explicitly factor bimodal financials, instead of with outlier lines. Could use mean since it will be skewed anyway.
salary: upper outlier bound
deferral_payments: upper outlier bound, and 0
total_payments: upper outlier or mean
loan_advances: about 4.0e+07 or 75th percentile
bonus: upper outlier
restricted_stock_deferred: upper outlier and 0
total_stock_value: 50th percentile and/or upper outlier, two binary features?
expenses: upper outlier or about 150000
exercised_stock_options: 50th, 75th, mean, upper outlier
other: mean or 500000
long_term_incentive: 3.0e+06
restricted_stock: 0, 0.5, 1 (1e7)
director_fees: about 75000

More-than-binary candidates can be turned into multiple features, one binary feature for each edge, and one poly for all levels. In fact, they all could be done this way, even if it's just the quartiles. We don't just want high recall of POIs; we also can say we want high-precision classification of nonPOIs. These features can enable the algorithm to apply more nuance. Feature selection can weed out insignificance, and PCA can compress redundancy. Maybe it will be a wash, but maybe not. I'm not going to run a comparison, though; I'll just do it.

For all categorical features, assign a category for nans (e.g. binary goes from (0, 1) to (0, 1, -1). Just use missingness features included in sklearn imputation
Counting whether a value is missing or not might bias the classification to nonPOIs.
Label some sparse-but-telling features as value present or not?
deferral_payments, loan_advances, restricted_stock_deferred, deferred_income

For each:
Quartiles (4-level)
Quintiles (5-level)
Deciles (10-level)
Binaries: 50th, 80th, 20th, 90th, 10th, Mean, Upper outlier, Lower outlier, Non-outlier, +/-, Missing value
Scaled without outliers, and scaled with outliers.
**???WON'T SCALING TO NON-OUTLYING RANGE DELIVER THE SAME INFORMATION AS FACTORING BY QUANTILES AND OUTLIER STATUS???**
Still want to account for +/-, missing
Outlier-free twin

For each pair:
Flag above or below linear regression.
Across domains: only compare ratios across domains (e.g. salary:total_payments::from_this_person_to_poi:from_messages), but maybe from_poi and to_poi can matter in relation to financials

Cluster features:
Bivariate cluster (2 clusters) of each pair.
Trivariate clusters (2 clusters) within subspaces (i.e. financial, email)
All multivariate clusters (2 clusters) within subspaces (i.e. financial, email), up to full space.
All clustering with and without outliers.
Remove nans rather than impute?

footnote labels

Features: engineer them all; let importance sort them out.

###### Features I probably won't engineer
Temporal proximity to fraud, in employment years, in emails

Job title

#### Imputation
Multivariate imputation before or afer categorical/scaling engineering?
Impute median for numeric? Mode for categorical? 0 for sparse-but-telling?
What kind of nulls does IterativeImputer accept?
Impute unique subspaces separately (fin_feat_cat, fin_feat_num, email_feat_cat, email_feat_num)


#### Coerced regression of target feature
Create a linear prediction of POI as float from salient features, and save as a feature.



#### Creating ratio features

In [5]:
### Start with all ratios, within respective subspaces (fin:fin, e:e)

###
### FIXME: Regression can't handle NaN input, though it could elsewhere.
###
# ### Also flag if ratio above it's regression.

# def flag_reg_error(y_sr, X_sr, index=None, pair_name=None,
#                    sk_reg=LinearRegression, **kwargs):
#     '''Flag whether a point falls above or below the regression.
#     Parameters:
#         y_sr: (pandas.Series, float) of y values.
#         x_sr: (pandas.Series, float) of x values.
#         index: (pandas.Series.index) to assign flagged_sr. If None (default),
#             use y_sr.index.
#         pair_name: (string) to name the return series ("__reg_error" appended);
#             If None (default), y_sr.name + "__" + x_sr.name.
#         sk_reg: (sklearn regression function) to fit and predict.
#         **kwargs: (list) keyword arguments to pass to sk_reg.
#     Return:
#         flagged_sr: (pandas.Series, bool) flags whether error is above 0.'''
    
#     flagged_sr = None #pd.Series()
    
#     if pair_name is None:
#         pair_name = y_sr.name + '__' + X_sr.name
#     if index is None:
#         index = y_sr.index
        
#     reg = sk_reg(**kwargs).fit(X=[X_sr], y=[y_sr])
#     pred = reg.predict(X=X_sr)
    
#     error_sr = y_sr - pred
#     flagged_sr = pd.Series(error_sr.apply(func=(lambda x: np.nan
#                                                 if x == np.nan or x == np.inf \
#                                                 else x > 0)))
#     flagged_sr.index = index
#     flagged_sr.name = pair_name + '__reg_error'
    
#     return flagged_sr

# flag_reg_error(y_sr=X_train['salary'], X_sr=X_train['bonus'])

def set_all_ratios(df, x_lst, y_lst, flag_error=False):
    '''Create new features from ratios of pairs of features.
    Optionally create feature that flags if cartesion point falls above or
    below regression.
    Parameters:
        df: (pandas.DataFrame) selected columns must be numeric, float best.
        x_lst: (list, str) names of columns to use as y in y/x.
        y_lst: (list, str) names of columns to use as x in y/x.
        flag_error: (bool) If True (default: False) create additional feature
            that flags whether point falls above regression.
    Return:
        ratio_subspace_df: (pandas.DataFrame) new features to add to dataset.
    '''
    ratio_subspace_df = None #pd.DataFrame()
    
    ### We want both ratios for each to get the 0s reciprocal to inf.
    for y in x_lst:
        for x in y_lst:
            if y != x:
                ratio_sr = df[y] / df[x]
                ratio_sr.name = y + '_DivBy_' + x
                ratio_subspace_df = pd.concat(objs=[ratio_subspace_df,
                                                    ratio_sr],
                                              axis=1)
                
###
### FIXME: finish flag_error (regression not accepting NaNs, does elsewhere)
###
#                 if flag_error:
#                     ratio_subspace_df = pd.concat(objs=[ratio_subspace_df,
#                                             flag_reg_error(y_sr=df[y],
#                                                            X_sr=df[x])],
#                                            axis=1)
    
    return ratio_subspace_df

In [6]:
### Add financial all ratios within subspaces to data sets.
pay_feats_divby_df = set_all_ratios(df=X_train, x_lst=pay_features,
                                    y_lst=pay_features)
stock_feats_divby_df = set_all_ratios(df=X_train, x_lst=stock_features,
                                    y_lst=stock_features)

### Only plausible email ratios (all reciprocals still, to get the 0s to infs):
to_lst = ['to_messages', 'from_poi_to_this_person', 'shared_receipt_with_poi']
from_lst = ['from_messages', 'from_this_person_to_poi']
email_to_divby_df = set_all_ratios(df=X_train, x_lst=to_lst, y_lst=to_lst)
email_from_divby_df = set_all_ratios(df=X_train, x_lst=from_lst,
                                     y_lst=from_lst)

X_train = pd.concat(objs=[X_train, pay_feats_divby_df, stock_feats_divby_df,
                          email_to_divby_df, email_from_divby_df], axis=1)


### Do for test set
pay_feats_divby_df = set_all_ratios(df=X_test, x_lst=pay_features,
                                    y_lst=pay_features)
stock_feats_divby_df = set_all_ratios(df=X_test, x_lst=stock_features,
                                    y_lst=stock_features)
email_to_divby_df = set_all_ratios(df=X_test, x_lst=to_lst, y_lst=to_lst)
email_from_divby_df = set_all_ratios(df=X_test, x_lst=from_lst, y_lst=from_lst)
X_test = pd.concat(objs=[X_test, pay_feats_divby_df, stock_feats_divby_df,
                          email_to_divby_df, email_from_divby_df], axis=1)

X_train.info()
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 86 entries, ECHOLS JOHN B to MURRAY JULIA H
Data columns (total 71 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   salary                                                 62 non-null     float64
 1   bonus                                                  54 non-null     float64
 2   long_term_incentive                                    40 non-null     float64
 3   deferred_income                                        22 non-null     float64
 4   deferral_payments                                      23 non-null     float64
 5   expenses                                               62 non-null     float64
 6   total_payments                                         72 non-null     float64
 7   exercised_stock_options                                64 non-null     float64
 8   restricted_stock                 

In [7]:
sorted(X_train.columns)
sorted(X_test.columns)

['bonus',
 'bonus_DivBy_deferral_payments',
 'bonus_DivBy_deferred_income',
 'bonus_DivBy_expenses',
 'bonus_DivBy_long_term_incentive',
 'bonus_DivBy_salary',
 'bonus_DivBy_total_payments',
 'deferral_payments',
 'deferral_payments_DivBy_bonus',
 'deferral_payments_DivBy_deferred_income',
 'deferral_payments_DivBy_expenses',
 'deferral_payments_DivBy_long_term_incentive',
 'deferral_payments_DivBy_salary',
 'deferral_payments_DivBy_total_payments',
 'deferred_income',
 'deferred_income_DivBy_bonus',
 'deferred_income_DivBy_deferral_payments',
 'deferred_income_DivBy_expenses',
 'deferred_income_DivBy_long_term_incentive',
 'deferred_income_DivBy_salary',
 'deferred_income_DivBy_total_payments',
 'exercised_stock_options',
 'exercised_stock_options_DivBy_restricted_stock',
 'exercised_stock_options_DivBy_total_stock_value',
 'expenses',
 'expenses_DivBy_bonus',
 'expenses_DivBy_deferral_payments',
 'expenses_DivBy_deferred_income',
 'expenses_DivBy_long_term_incentive',
 'expenses_DivB

['bonus',
 'bonus_DivBy_deferral_payments',
 'bonus_DivBy_deferred_income',
 'bonus_DivBy_expenses',
 'bonus_DivBy_long_term_incentive',
 'bonus_DivBy_salary',
 'bonus_DivBy_total_payments',
 'deferral_payments',
 'deferral_payments_DivBy_bonus',
 'deferral_payments_DivBy_deferred_income',
 'deferral_payments_DivBy_expenses',
 'deferral_payments_DivBy_long_term_incentive',
 'deferral_payments_DivBy_salary',
 'deferral_payments_DivBy_total_payments',
 'deferred_income',
 'deferred_income_DivBy_bonus',
 'deferred_income_DivBy_deferral_payments',
 'deferred_income_DivBy_expenses',
 'deferred_income_DivBy_long_term_incentive',
 'deferred_income_DivBy_salary',
 'deferred_income_DivBy_total_payments',
 'exercised_stock_options',
 'exercised_stock_options_DivBy_restricted_stock',
 'exercised_stock_options_DivBy_total_stock_value',
 'expenses',
 'expenses_DivBy_bonus',
 'expenses_DivBy_deferral_payments',
 'expenses_DivBy_deferred_income',
 'expenses_DivBy_long_term_incentive',
 'expenses_DivB

In [8]:
### Set all np.inf to np.nan.
X_train = X_train.apply(func=(lambda col: \
         col.apply(func=(lambda x: np.nan if abs(x) == abs(np.inf) else x))))
X_test = X_test.apply(func=(lambda col: \
         col.apply(func=(lambda x: np.nan if abs(x) == abs(np.inf) else x))))

In [9]:
sorted(X_train.columns)
sorted(X_test.columns)

['bonus',
 'bonus_DivBy_deferral_payments',
 'bonus_DivBy_deferred_income',
 'bonus_DivBy_expenses',
 'bonus_DivBy_long_term_incentive',
 'bonus_DivBy_salary',
 'bonus_DivBy_total_payments',
 'deferral_payments',
 'deferral_payments_DivBy_bonus',
 'deferral_payments_DivBy_deferred_income',
 'deferral_payments_DivBy_expenses',
 'deferral_payments_DivBy_long_term_incentive',
 'deferral_payments_DivBy_salary',
 'deferral_payments_DivBy_total_payments',
 'deferred_income',
 'deferred_income_DivBy_bonus',
 'deferred_income_DivBy_deferral_payments',
 'deferred_income_DivBy_expenses',
 'deferred_income_DivBy_long_term_incentive',
 'deferred_income_DivBy_salary',
 'deferred_income_DivBy_total_payments',
 'exercised_stock_options',
 'exercised_stock_options_DivBy_restricted_stock',
 'exercised_stock_options_DivBy_total_stock_value',
 'expenses',
 'expenses_DivBy_bonus',
 'expenses_DivBy_deferral_payments',
 'expenses_DivBy_deferred_income',
 'expenses_DivBy_long_term_incentive',
 'expenses_DivB

['bonus',
 'bonus_DivBy_deferral_payments',
 'bonus_DivBy_deferred_income',
 'bonus_DivBy_expenses',
 'bonus_DivBy_long_term_incentive',
 'bonus_DivBy_salary',
 'bonus_DivBy_total_payments',
 'deferral_payments',
 'deferral_payments_DivBy_bonus',
 'deferral_payments_DivBy_deferred_income',
 'deferral_payments_DivBy_expenses',
 'deferral_payments_DivBy_long_term_incentive',
 'deferral_payments_DivBy_salary',
 'deferral_payments_DivBy_total_payments',
 'deferred_income',
 'deferred_income_DivBy_bonus',
 'deferred_income_DivBy_deferral_payments',
 'deferred_income_DivBy_expenses',
 'deferred_income_DivBy_long_term_incentive',
 'deferred_income_DivBy_salary',
 'deferred_income_DivBy_total_payments',
 'exercised_stock_options',
 'exercised_stock_options_DivBy_restricted_stock',
 'exercised_stock_options_DivBy_total_stock_value',
 'expenses',
 'expenses_DivBy_bonus',
 'expenses_DivBy_deferral_payments',
 'expenses_DivBy_deferred_income',
 'expenses_DivBy_long_term_incentive',
 'expenses_DivB

In [10]:
### Remove all features containing less than 30 training observations.
drop_lst = list(X_train.count().loc[X_train.count() \
                                    < .3 * len(X_train.index)].index)
print(drop_lst)
X_train.drop(columns=drop_lst, inplace=True)
X_test.drop(columns=drop_lst, inplace=True)

pay_feats_divby_lst = [feat for feat in list(pay_feats_divby_df.columns) \
                       if not feat in drop_lst]
stock_feats_divby_lst = [feat for feat in list(stock_feats_divby_df.columns) \
                       if not feat in drop_lst]
email_feats_divby_lst = [feat for feat in list(email_to_divby_df.columns) \
                       if not feat in drop_lst] \
                    + [feat for feat in list(email_from_divby_df.columns) \
                       if not feat in drop_lst]
fin_features = [feat for feat in fin_features if feat not in drop_lst]
pay_features = [feat for feat in pay_features if feat not in drop_lst]
stock_features = [feat for feat in stock_features if feat not in drop_lst]
email_features = [feat for feat in email_features if feat not in drop_lst]
features_list = [feat for feat in features_list if feat not in drop_lst]

X_train.info()
X_test.info()

['deferred_income', 'deferral_payments', 'salary_DivBy_deferred_income', 'salary_DivBy_deferral_payments', 'bonus_DivBy_deferred_income', 'bonus_DivBy_deferral_payments', 'long_term_incentive_DivBy_deferred_income', 'long_term_incentive_DivBy_deferral_payments', 'deferred_income_DivBy_salary', 'deferred_income_DivBy_bonus', 'deferred_income_DivBy_long_term_incentive', 'deferred_income_DivBy_deferral_payments', 'deferred_income_DivBy_expenses', 'deferred_income_DivBy_total_payments', 'deferral_payments_DivBy_salary', 'deferral_payments_DivBy_bonus', 'deferral_payments_DivBy_long_term_incentive', 'deferral_payments_DivBy_deferred_income', 'deferral_payments_DivBy_expenses', 'deferral_payments_DivBy_total_payments', 'expenses_DivBy_deferred_income', 'expenses_DivBy_deferral_payments', 'total_payments_DivBy_deferred_income', 'total_payments_DivBy_deferral_payments']
<class 'pandas.core.frame.DataFrame'>
Index: 86 entries, ECHOLS JOHN B to MURRAY JULIA H
Data columns (total 47 columns):
 # 

#### Create quantile / outlier flags

In [11]:
# # def set_quant_flag_col(feat_col, high_q, low_q, q_name):
# #     '''Create a series that flags each value of the input series as inside a
# #     range, outside of it, or np.nan.
# #     Parameters:
# #         feat_col: (pandas.Series, float) series to flag
# #         up_q: (float) upper limit of range, exclusive
# #         down_q: (float) lower limite of range, inclusive
# #     Return:
# #         flag_col: (panda.Series, bool) parallel series with flags
# #     '''
# #     flag_col = None # pd.Series()
    
# #     flag_col = feat_col.apply(func=(lambda x:
# #                                     x >= low_q and x < (high_q)
# #                                     if not (np.isnan(x)) else np.nan))
# #     flag_col.name = feat_col.name + '_' + q_name
# # #     print('flag_col:', flag_col.name)
# # #     print('flag_col type:', type(flag_col))
# #     return flag_col

# def create_quant_flag_cols(feat_col, quant_col, num_q_levels=4, q_name='4'):
#     '''Create a flagged column for each quantile level of the input feature.
#     Parameters:
#         feat_col: (pandas.Series, float) feature to flag.
#         quant_col: (pandas.Series, float) feature to set quantiles;
#             X_train if flagging X_test.
#         num_q_levels: (int) number of quantiles to use.
#     Return:
#         flagged_subspace_df: (pandas.DataFrame, bool) dataset of new features
#     '''
#     flagged_subspace_df = None # pd.DataFrame()
    
#     qn = np.quantile(a=quant_col.dropna(),
#                      q=np.arange(start=0, stop=1,
#                                  step=(1 / (num_q_levels -1))))
#     q_levels = np.append(arr=[-np.inf], values=qn)
#     q_levels = np.append(arr=q_levels, values=np.inf)
# #     print('q_levels:', q_levels)
    
#     ### Create a new flagged feature for each quantile,
#     ### and add it to the return dataframe.
#     i = 0
#     for low_q in q_levels:
#         i += 1
#         q_name1 = feat_col.name + '_' + str(i) + '_of_' + q_name + '_quant'
# #         print('q_name1:', q_name1)
#         flagged_col = feat_col.apply(func=(lambda x: np.nan if np.isnan(x)
#                                            else x > low_q and x <= q_levels[1]))
#         flagged_col.name = q_name1
# #         print('flagged_col:', flagged_col)
#         flagged_subspace_df = pd.concat(objs=[flagged_subspace_df,
#                                               flagged_col],
#                                         axis=1)

# ### Incorrectly sets quantiles, and we don't need another function.
# #     i = 0
# #     last = len(q_levels) - 1
# #     for low_q in q_levels:
# #         if i == last:
# #             high_q = np.inf
# #         else:
# #             high_q = q_levels[i+1]
# #         q_name1 = str(i+1) + '_of_' + q_name + '_quant'
# # #         print('q_name1:', q_name1)
# #         flagged_subspace_df = pd.concat(objs=[flagged_subspace_df,
# #                                               set_quant_flag_col(
# #                                                             feat_col=feat_col,
# #                                                             high_q=high_q,
# #                                                             low_q=low_q,
# #                                                             q_name=q_name1)],
# #                                         axis=1)
    
# #         i += 1

# #     print('flagged_subspace_df.info():', flagged_subspace_df.info())

#     return flagged_subspace_df

# def create_outlier_flag_cols(feat_col, quant_col):
#     '''Create a flagged column for each end of outliership, using 1.5*IQR.
#     Parameters:
#         feat_col: (pandas.Series) column to flag.
#         quant_col: (pandas.Series) column to construct outlier bounds.
#                     Always X_train.'''
#     flagged_subspace_df = None # pd.DataFrame()
    
#     quarts = np.quantile(a=quant_col.dropna(),
#                          q=np.arange(start=0, stop=1, step=1/3))
#     IQR = quarts[2] - quarts[0]
    
#     low_out_sr = feat_col.apply(func=(lambda x: np.nan if np.isnan(x)
#                                       else x < quarts[1] - (1.5 * IQR)))
#     low_out_sr.name = feat_col.name + '_low_out'
#     high_out_sr = feat_col.apply(func=(lambda x: np.nan if np.isnan(x)
#                                        else x > quarts[1] + (1.5 * IQR)))
#     high_out_sr.name = feat_col.name + '_high_out'
    
#     flagged_subspace_df = pd.concat(objs=[flagged_subspace_df,
#                                           low_out_sr, high_out_sr],
#                                     axis=1)
    
# #     print('flagged_subspace_df.info():', flagged_subspace_df.info())
    
#     return flagged_subspace_df

# def create_all_quant_flag_cols(feat_col, quant_col, quants=[4, 5, 10],
#                                outs=True):
#     '''Create flagged features for each number of quant levels.
#     And, a flagged column for each end of outliership.
#     Parameters:
#         feat_col: (pandas.Series, float) feature to flag.
#         quant_col: (pandas.Series, float) feature to set quantiles;
#             Always X_train.
#         quants: (list, int) quantiles to use.
#             [4, 5, 10] will create 19 new features, i.e. four quartile flags,
#             five quintile flags, and ten decile flags.
#     Return:
#         flagged_subspace_df: (pandas.DataFrame, float) dataset of new features.
#     '''
#     flagged_subspace_df = None # pd.DataFrame()
    
#     for quant in quants:
#         q_name = str(quant)
# #         print('q_name:', q_name)
#         flagged_subspace_df = pd.concat(objs=[flagged_subspace_df,
#                                               create_quant_flag_cols(\
#                                                 feat_col=feat_col,
#                                                 quant_col=quant_col,
#                                                 num_q_levels=quant,
#                                                 q_name=q_name)],
#                                         axis=1)
    
#     if outs:
#         flagged_subspace_df = pd.concat(objs=[flagged_subspace_df,
#                                               create_outlier_flag_cols(\
#                                                 feat_col=feat_col,
#                                                 quant_col=quant_col)],
#                                         axis=1)
        
# #     print('flagged_subspace_df.info():', flagged_subspace_df.info())
    
#     return flagged_subspace_df

# def quant_flag_all(df, quant_df, quants=[4, 5, 10]):
#     '''For each feature, create flagged features for each number of quant
#     levels.
#     Parameters:
#         df: (pandas.DataFrame, float) dataset with all features to flag.
#         quant_df: (pandas.DataFrame, float) dataset to set quantiles;
#             X_train if flagging X_test.
#         quants: (list, int) quantiles to use.
#             [4, 5, 10] will create 19 new features per feature,
#             i.e. four quartile flags, five quintile flags, and ten decile
#             flags.
#     Return:
#         flagged_subspace_df: (pandas.DataFrame, float) dataset of new features.
#     '''
#     flagged_subspace_df = None #pd.DataFrame()
    
#     for feat in df.columns:
#         flagged_subspace_df = pd.concat(objs=[flagged_subspace_df,
#                                               create_all_quant_flag_cols(\
#                                                 feat_col=df[feat],
#                                                 quant_col=quant_df[feat],
#                                                 quants=quants)],
#                                         axis=1)
# #     print('flagged_subspace_df type:', type(flagged_subspace_df))
#     return flagged_subspace_df

# X_train.info()

# quant_flags_df = quant_flag_all(df=X_train, quant_df=X_train)
# # print('quant_flags_df type:', type(quant_flags_df))
# X_train = pd.concat(objs=[X_train, quant_flags_df.astype(float)],
#                     axis=1)

# X_train.head()
# X_train.info()

# X_test.info()

# quant_flags_df = quant_flag_all(df=X_test, quant_df=X_train)
# # print('quant_flags_df type:', type(quant_flags_df))
# X_test = pd.concat(objs=[X_test, quant_flags_df.astype(float)], axis=1)

# X_test.head()
# X_test.info()

# quant_flags_lst = list(quant_flags_df.columns)
# X_train[quant_flags_lst].head()

In [12]:
def quant_flag_all(df, quant_df, quants=[4, 5, 10]):
    '''For each feature, create a categorical feature for each quantile and for outliers.
    Parameters:
        df: (pandas.DataFrame, float) dataset with all features to flag.
        quant_df: (pandas.DataFrame, float) dataset to set quantiles;
            X_train if flagging X_test.
        quants: (list, int) quantiles to use.
            [4, 5, 10] will create 3 new features per feature,
            i.e. a quartile factor, a quintile factor, and a decile factor.
    Return:
        flagged_subspace_df: (pandas.DataFrame, float) dataset of new features.
    '''
    flagged_subspace_df = None #pd.DataFrame()
    
    for feat in df.columns:
        ### Get outlier values, create empty series.
        quarts = np.quantile(a=quant_df[feat].dropna(), q=np.arange(0, 1, 1/3))
        IQRmult = 1.5 * (quarts[2] - quarts[0])
        outs = [(quarts[0] - IQRmult), (quarts[2] + IQRmult)]
        out_sr = pd.Series(dtype=int, name=(feat + '_outliers'))
        
        ### Create quant ranges, empty df with column names.
        q_ranges_dict = {}
        q_subspace_df = pd.DataFrame(dtype=float, index=df.index)
        for quant in quants:
            q = np.arange(start=0, stop=1, step=(1/quant))
            qn = np.quantile(a=quant_df[feat].dropna(), q=q)
            q_ranges = np.append(arr=(np.append(arr=[-np.inf],
                                                values=qn)),
                                 values=[np.inf])
            q_name = feat + '_q_' + str(quant)
            q_ranges_dict[q_name] = q_ranges
            q_sr = pd.Series(dtype=float, name=q_name)
            q_subspace_df = pd.concat(objs=[q_subspace_df, q_sr], axis=1)
        
        ### Not encapsulated in order to save a little compute.
        for i, x in enumerate(df[feat]):
            if not np.isnan(x):
                ### Populate outlier column.
                if x < outs[0]:
                    out_sr[df[feat].index[i]] = -1
                elif x > outs[1]:
                    out_sr[df[feat].index[i]] = 1
                else:
                    out_sr[df[feat].index[i]] = 0

                ### Populate each quant column.
                for q_name, q_ranges in q_ranges_dict.items():
                    for j, low_q in enumerate(q_ranges):
                        if (x > low_q) and (x <= q_ranges[j + 1]):
                            q_subspace_df[q_name][df.index[i]] = j + 1
                            break
        
        flagged_subspace_df = pd.concat(objs=[flagged_subspace_df, out_sr,
                                              q_subspace_df],
                                        axis=1)

    return flagged_subspace_df


X_train
X_train.info()
quant_flags_df = quant_flag_all(df=X_train, quant_df=X_train)
X_train = pd.concat(objs=[X_train, quant_flags_df.astype(float)],
                    axis=1)
X_train
X_train.info()

X_test
X_test.info()
quant_flags_df = quant_flag_all(df=X_test, quant_df=X_train)
X_test = pd.concat(objs=[X_test, quant_flags_df.astype(float)], axis=1)
X_test
X_test.info()

,salary,bonus,long_term_incentive,expenses,total_payments,exercised_stock_options,restricted_stock,total_stock_value,to_messages,from_poi_to_this_person,...,total_stock_value_DivBy_exercised_stock_options,total_stock_value_DivBy_restricted_stock,to_messages_DivBy_from_poi_to_this_person,to_messages_DivBy_shared_receipt_with_poi,from_poi_to_this_person_DivBy_to_messages,from_poi_to_this_person_DivBy_shared_receipt_with_poi,shared_receipt_with_poi_DivBy_to_messages,shared_receipt_with_poi_DivBy_from_poi_to_this_person,from_messages_DivBy_from_this_person_to_poi,from_this_person_to_poi_DivBy_from_messages
ECHOLS JOHN B,182245.0,200000.0,1920000.0,21530.0,2692324.0,601438.0,407503.0,1008941.0,NaN,NaN,...,1.677548,2.475911,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
KAMINSKI WINCENTY J,275101.0,400000.0,323466.0,83585.0,1086821.0,850010.0,126027.0,976037.0,4607.0,41.0,...,1.148265,7.744666,112.365854,7.902230,0.008900,0.070326,0.126547,14.219512,46.783626,0.021375
BOWEN JR RAYMOND M,278601.0,1350000.0,974293.0,65907.0,2669589.0,NaN,252055.0,252055.0,1858.0,140.0,...,NaN,1.000000,13.271429,1.166353,0.075350,0.087884,0.857374,11.378571,1.800000,0.555556
PIPER GREGORY F,197091.0,400000.0,NaN,43057.0,1737629.0,880290.0,409554.0,880290.0,1238.0,61.0,...,1.000000,2.149387,20.295082,1.668464,0.049273,0.082210,0.599354,12.163934,4.625000,0.216216
HAYES ROBERT E,NaN,NaN,NaN,NaN,NaN,NaN,151418.0,151418.0,504.0,16.0,...,NaN,1.000000,31.500000,10.080000,0.031746,0.320000,0.099206,3.125000,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HIRKO JOSEPH,NaN,NaN,NaN,77978.0,91093.0,30766064.0,NaN,30766064.0,NaN,NaN,...,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GIBBS DANA R,NaN,NaN,461912.0,NaN,966522.0,2218275.0,NaN,2218275.0,169.0,0.0,...,1.000000,NaN,NaN,7.347826,0.000000,0.000000,0.136095,NaN,NaN,0.000000
SHARP VICTORIA T,248146.0,600000.0,422158.0,116337.0,1576511.0,281073.0,213063.0,494136.0,3136.0,24.0,...,1.758034,2.319201,130.666667,1.266048,0.007653,0.009689,0.789860,103.208333,22.666667,0.044118
PIRO JIM,NaN,NaN,NaN,NaN,NaN,NaN,47304.0,47304.0,58.0,0.0,...,NaN,1.000000,NaN,19.333333,0.000000,0.000000,0.051724,NaN,16.000000,0.062500


<class 'pandas.core.frame.DataFrame'>
Index: 86 entries, ECHOLS JOHN B to MURRAY JULIA H
Data columns (total 47 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   salary                                                 62 non-null     float64
 1   bonus                                                  54 non-null     float64
 2   long_term_incentive                                    40 non-null     float64
 3   expenses                                               62 non-null     float64
 4   total_payments                                         72 non-null     float64
 5   exercised_stock_options                                64 non-null     float64
 6   restricted_stock                                       72 non-null     float64
 7   total_stock_value                                      81 non-null     float64
 8   to_messages                      

,salary,bonus,long_term_incentive,expenses,total_payments,exercised_stock_options,restricted_stock,total_stock_value,to_messages,from_poi_to_this_person,...,shared_receipt_with_poi_DivBy_from_poi_to_this_person_q_5,shared_receipt_with_poi_DivBy_from_poi_to_this_person_q_10,from_messages_DivBy_from_this_person_to_poi_outliers,from_messages_DivBy_from_this_person_to_poi_q_4,from_messages_DivBy_from_this_person_to_poi_q_5,from_messages_DivBy_from_this_person_to_poi_q_10,from_this_person_to_poi_DivBy_from_messages_outliers,from_this_person_to_poi_DivBy_from_messages_q_4,from_this_person_to_poi_DivBy_from_messages_q_5,from_this_person_to_poi_DivBy_from_messages_q_10
ECHOLS JOHN B,182245.0,200000.0,1920000.0,21530.0,2692324.0,601438.0,407503.0,1008941.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
KAMINSKI WINCENTY J,275101.0,400000.0,323466.0,83585.0,1086821.0,850010.0,126027.0,976037.0,4607.0,41.0,...,3.0,5.0,1.0,5.0,6.0,11.0,0.0,3.0,3.0,5.0
BOWEN JR RAYMOND M,278601.0,1350000.0,974293.0,65907.0,2669589.0,NaN,252055.0,252055.0,1858.0,140.0,...,3.0,4.0,0.0,2.0,2.0,2.0,0.0,5.0,6.0,11.0
PIPER GREGORY F,197091.0,400000.0,NaN,43057.0,1737629.0,880290.0,409554.0,880290.0,1238.0,61.0,...,3.0,4.0,0.0,4.0,4.0,7.0,0.0,4.0,5.0,8.0
HAYES ROBERT E,NaN,NaN,NaN,NaN,NaN,NaN,151418.0,151418.0,504.0,16.0,...,1.0,1.0,NaN,NaN,NaN,NaN,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HIRKO JOSEPH,NaN,NaN,NaN,77978.0,91093.0,30766064.0,NaN,30766064.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GIBBS DANA R,NaN,NaN,461912.0,NaN,966522.0,2218275.0,NaN,2218275.0,169.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,1.0
SHARP VICTORIA T,248146.0,600000.0,422158.0,116337.0,1576511.0,281073.0,213063.0,494136.0,3136.0,24.0,...,6.0,11.0,0.0,5.0,6.0,10.0,0.0,3.0,4.0,6.0
PIRO JIM,NaN,NaN,NaN,NaN,NaN,NaN,47304.0,47304.0,58.0,0.0,...,NaN,NaN,0.0,4.0,5.0,9.0,0.0,3.0,4.0,6.0


<class 'pandas.core.frame.DataFrame'>
Index: 86 entries, ECHOLS JOHN B to MURRAY JULIA H
Columns: 235 entries, salary to from_this_person_to_poi_DivBy_from_messages_q_10
dtypes: float64(235)
memory usage: 158.6+ KB


,salary,bonus,long_term_incentive,expenses,total_payments,exercised_stock_options,restricted_stock,total_stock_value,to_messages,from_poi_to_this_person,...,total_stock_value_DivBy_exercised_stock_options,total_stock_value_DivBy_restricted_stock,to_messages_DivBy_from_poi_to_this_person,to_messages_DivBy_shared_receipt_with_poi,from_poi_to_this_person_DivBy_to_messages,from_poi_to_this_person_DivBy_shared_receipt_with_poi,shared_receipt_with_poi_DivBy_to_messages,shared_receipt_with_poi_DivBy_from_poi_to_this_person,from_messages_DivBy_from_this_person_to_poi,from_this_person_to_poi_DivBy_from_messages
WESTFAHL RICHARD K,63744.0,NaN,256191.0,51870.0,762135.0,NaN,384930.0,384930.0,NaN,NaN,...,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SHELBY REX,211844.0,200000.0,NaN,22884.0,2003885.0,1624396.0,869220.0,2493616.0,225.0,13.0,...,1.535104,2.868797,17.307692,2.472527,0.057778,0.142857,0.404444,7.000000,2.785714,0.358974
DEFFNER JOSEPH M,206121.0,600000.0,335349.0,41626.0,1208649.0,17378.0,141833.0,159211.0,714.0,115.0,...,9.161641,1.122524,6.208696,1.293478,0.161064,0.208333,0.773109,4.800000,18.500000,0.054054
WASAFF GEORGE,259996.0,325000.0,200000.0,NaN,1034395.0,1668260.0,388167.0,2056427.0,400.0,22.0,...,1.232678,5.297789,18.181818,1.186944,0.055000,0.065282,0.842500,15.318182,4.285714,0.233333
HANNON KEVIN P,243293.0,1500000.0,1617011.0,34039.0,288682.0,5538001.0,853064.0,6391065.0,1045.0,32.0,...,1.154038,7.491894,32.656250,1.009662,0.030622,0.030918,0.990431,32.343750,1.523810,0.656250
GOLD JOSEPH,272880.0,750000.0,304805.0,NaN,2146973.0,436515.0,441096.0,877611.0,NaN,NaN,...,2.010494,1.989615,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HERMANN ROBERT J,262663.0,700000.0,150000.0,48357.0,1297461.0,187500.0,480632.0,668132.0,NaN,NaN,...,3.563371,1.390111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CARTER REBECCA C,261809.0,300000.0,75000.0,NaN,477557.0,NaN,307301.0,0.0,312.0,29.0,...,NaN,0.000000,10.758621,1.591837,0.092949,0.147959,0.628205,6.758621,2.142857,0.466667
HICKERSON GARY J,211788.0,1700000.0,69223.0,98849.0,2081796.0,NaN,441096.0,441096.0,1320.0,40.0,...,NaN,1.000000,33.000000,1.466667,0.030303,0.044444,0.681818,22.500000,27.000000,0.037037
LAY KENNETH L,1072321.0,7000000.0,3600000.0,99832.0,103559793.0,34348384.0,14761694.0,49110078.0,4273.0,123.0,...,1.429764,3.326859,34.739837,1.772294,0.028785,0.051016,0.564241,19.601626,2.250000,0.444444


<class 'pandas.core.frame.DataFrame'>
Index: 37 entries, WESTFAHL RICHARD K to PRENTICE JAMES
Data columns (total 47 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   salary                                                 31 non-null     float64
 1   bonus                                                  27 non-null     float64
 2   long_term_incentive                                    25 non-null     float64
 3   expenses                                               24 non-null     float64
 4   total_payments                                         37 non-null     float64
 5   exercised_stock_options                                27 non-null     float64
 6   restricted_stock                                       32 non-null     float64
 7   total_stock_value                                      36 non-null     float64
 8   to_messages                 

,salary,bonus,long_term_incentive,expenses,total_payments,exercised_stock_options,restricted_stock,total_stock_value,to_messages,from_poi_to_this_person,...,shared_receipt_with_poi_DivBy_from_poi_to_this_person_q_5,shared_receipt_with_poi_DivBy_from_poi_to_this_person_q_10,from_messages_DivBy_from_this_person_to_poi_outliers,from_messages_DivBy_from_this_person_to_poi_q_4,from_messages_DivBy_from_this_person_to_poi_q_5,from_messages_DivBy_from_this_person_to_poi_q_10,from_this_person_to_poi_DivBy_from_messages_outliers,from_this_person_to_poi_DivBy_from_messages_q_4,from_this_person_to_poi_DivBy_from_messages_q_5,from_this_person_to_poi_DivBy_from_messages_q_10
WESTFAHL RICHARD K,63744.0,NaN,256191.0,51870.0,762135.0,NaN,384930.0,384930.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SHELBY REX,211844.0,200000.0,NaN,22884.0,2003885.0,1624396.0,869220.0,2493616.0,225.0,13.0,...,2.0,3.0,0.0,2.0,2.0,3.0,0.0,5.0,6.0,10.0
DEFFNER JOSEPH M,206121.0,600000.0,335349.0,41626.0,1208649.0,17378.0,141833.0,159211.0,714.0,115.0,...,2.0,2.0,0.0,5.0,5.0,9.0,0.0,3.0,4.0,6.0
WASAFF GEORGE,259996.0,325000.0,200000.0,NaN,1034395.0,1668260.0,388167.0,2056427.0,400.0,22.0,...,3.0,5.0,0.0,3.0,4.0,6.0,0.0,4.0,5.0,9.0
HANNON KEVIN P,243293.0,1500000.0,1617011.0,34039.0,288682.0,5538001.0,853064.0,6391065.0,1045.0,32.0,...,5.0,9.0,0.0,2.0,2.0,2.0,1.0,5.0,6.0,11.0
GOLD JOSEPH,272880.0,750000.0,304805.0,NaN,2146973.0,436515.0,441096.0,877611.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HERMANN ROBERT J,262663.0,700000.0,150000.0,48357.0,1297461.0,187500.0,480632.0,668132.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CARTER REBECCA C,261809.0,300000.0,75000.0,NaN,477557.0,NaN,307301.0,0.0,312.0,29.0,...,2.0,3.0,0.0,2.0,2.0,3.0,0.0,5.0,6.0,10.0
HICKERSON GARY J,211788.0,1700000.0,69223.0,98849.0,2081796.0,NaN,441096.0,441096.0,1320.0,40.0,...,4.0,7.0,1.0,5.0,6.0,10.0,0.0,3.0,3.0,5.0
LAY KENNETH L,1072321.0,7000000.0,3600000.0,99832.0,103559793.0,34348384.0,14761694.0,49110078.0,4273.0,123.0,...,4.0,7.0,0.0,2.0,2.0,3.0,0.0,5.0,6.0,10.0


<class 'pandas.core.frame.DataFrame'>
Index: 37 entries, WESTFAHL RICHARD K to PRENTICE JAMES
Columns: 235 entries, salary to from_this_person_to_poi_DivBy_from_messages_q_10
dtypes: float64(235)
memory usage: 68.2+ KB


In [13]:
quant_flags_lst = [feat for feat in list(quant_flags_df.columns)
                   if feat.find('_q_') != -1]
out_flags_lst = [feat for feat in quant_flags_df.columns
                 if feat not in quant_flags_lst]
X_train[quant_flags_lst]
X_train[out_flags_lst]

# test_df = X_train
# test_df.info()

# quant_flags_df = quant_flag_all(df=test_df, quant_df=test_df)
# test_df = pd.concat(objs=[test_df, quant_flags_df],
#                     axis=1)

# test_df.head()
# test_df.info()

# quant_flags_lst = list(quant_flags_df.columns)
# test_df[quant_flags_lst]

,salary_q_4,salary_q_5,salary_q_10,bonus_q_4,bonus_q_5,bonus_q_10,long_term_incentive_q_4,long_term_incentive_q_5,long_term_incentive_q_10,expenses_q_4,...,shared_receipt_with_poi_DivBy_to_messages_q_10,shared_receipt_with_poi_DivBy_from_poi_to_this_person_q_4,shared_receipt_with_poi_DivBy_from_poi_to_this_person_q_5,shared_receipt_with_poi_DivBy_from_poi_to_this_person_q_10,from_messages_DivBy_from_this_person_to_poi_q_4,from_messages_DivBy_from_this_person_to_poi_q_5,from_messages_DivBy_from_this_person_to_poi_q_10,from_this_person_to_poi_DivBy_from_messages_q_4,from_this_person_to_poi_DivBy_from_messages_q_5,from_this_person_to_poi_DivBy_from_messages_q_10
ECHOLS JOHN B,2.0,2.0,3.0,2.0,2.0,2.0,5.0,6.0,11.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
KAMINSKI WINCENTY J,4.0,5.0,8.0,2.0,2.0,3.0,3.0,3.0,5.0,5.0,...,3.0,3.0,3.0,5.0,5.0,6.0,11.0,3.0,3.0,5.0
BOWEN JR RAYMOND M,4.0,5.0,8.0,5.0,6.0,10.0,4.0,5.0,9.0,4.0,...,9.0,2.0,3.0,4.0,2.0,2.0,2.0,5.0,6.0,11.0
PIPER GREGORY F,2.0,3.0,4.0,2.0,2.0,3.0,NaN,NaN,NaN,3.0,...,6.0,3.0,3.0,4.0,4.0,4.0,7.0,4.0,5.0,8.0
HAYES ROBERT E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,1.0,1.0,1.0,NaN,NaN,NaN,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HIRKO JOSEPH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GIBBS DANA R,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0,7.0,NaN,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
SHARP VICTORIA T,3.0,4.0,6.0,3.0,3.0,5.0,3.0,4.0,6.0,5.0,...,8.0,5.0,6.0,11.0,5.0,6.0,10.0,3.0,4.0,6.0
PIRO JIM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,NaN,NaN,NaN,4.0,5.0,9.0,3.0,4.0,6.0


,salary_outliers,bonus_outliers,long_term_incentive_outliers,expenses_outliers,total_payments_outliers,exercised_stock_options_outliers,restricted_stock_outliers,total_stock_value_outliers,to_messages_outliers,from_poi_to_this_person_outliers,...,total_stock_value_DivBy_exercised_stock_options_outliers,total_stock_value_DivBy_restricted_stock_outliers,to_messages_DivBy_from_poi_to_this_person_outliers,to_messages_DivBy_shared_receipt_with_poi_outliers,from_poi_to_this_person_DivBy_to_messages_outliers,from_poi_to_this_person_DivBy_shared_receipt_with_poi_outliers,shared_receipt_with_poi_DivBy_to_messages_outliers,shared_receipt_with_poi_DivBy_from_poi_to_this_person_outliers,from_messages_DivBy_from_this_person_to_poi_outliers,from_this_person_to_poi_DivBy_from_messages_outliers
ECHOLS JOHN B,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
KAMINSKI WINCENTY J,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
BOWEN JR RAYMOND M,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,1.0,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PIPER GREGORY F,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HAYES ROBERT E,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,...,NaN,0.0,0.0,1.0,0.0,1.0,0.0,0.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HIRKO JOSEPH,NaN,NaN,NaN,0.0,0.0,1.0,NaN,1.0,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GIBBS DANA R,NaN,NaN,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,...,0.0,NaN,NaN,1.0,0.0,0.0,0.0,NaN,NaN,0.0
SHARP VICTORIA T,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
PIRO JIM,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,...,NaN,0.0,NaN,1.0,0.0,0.0,0.0,NaN,0.0,0.0


In [14]:
### For each feature, create a feature that flags whether x > 0.
### Retain np.nans.

def flag_signs(df, suffix=''):
    '''For each feature, create a feature that flags whether x > 0.
    Retain np.nans.
    Parameters:
        df: (pandas.DataFrame)
        suffix: (str) if not empty (default: empty string), append suffix to
            column names.
    Returns
        flagged_df: (pandas.DataFrame, bool) of same shape as df.
    '''
    flagged_df = None #pd.DataFrame
    
    flagged_df = df.apply(func=(lambda col: col.apply(func=(lambda x: np.nan
                                                if np.isnan(x) else x > 0))))
    
    if suffix:
        new_cols_lst = []
        for col in flagged_df.columns:
            new_cols_lst.append(col + suffix)
        flagged_df.columns = new_cols_lst
        
    return flagged_df
    
    
# feats_lst = fin_features + email_features + pay_feats_divby_lst \
#             + stock_feats_divby_lst + email_feats_divby_lst
to_flag_lst = fin_features + email_features + pay_feats_divby_lst \
            + stock_feats_divby_lst + email_feats_divby_lst

sign_flags_df = flag_signs(df=X_train[to_flag_lst], suffix='_sign')
X_train = pd.concat(objs=[X_train, sign_flags_df], axis=1)
sign_flags_df = flag_signs(df=X_test[to_flag_lst], suffix='_sign')
X_test = pd.concat(objs=[X_test, sign_flags_df], axis=1)

sign_flags_lst = list(sign_flags_df.columns)

X_train.info()
X_test.info()
X_train[sign_flags_lst].head()

<class 'pandas.core.frame.DataFrame'>
Index: 86 entries, ECHOLS JOHN B to MURRAY JULIA H
Columns: 282 entries, salary to from_this_person_to_poi_DivBy_from_messages_sign
dtypes: float64(235), object(47)
memory usage: 190.1+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 37 entries, WESTFAHL RICHARD K to PRENTICE JAMES
Columns: 282 entries, salary to from_this_person_to_poi_DivBy_from_messages_sign
dtypes: bool(1), float64(235), object(46)
memory usage: 81.6+ KB


,salary_sign,bonus_sign,long_term_incentive_sign,expenses_sign,total_payments_sign,exercised_stock_options_sign,restricted_stock_sign,total_stock_value_sign,to_messages_sign,from_poi_to_this_person_sign,...,total_stock_value_DivBy_exercised_stock_options_sign,total_stock_value_DivBy_restricted_stock_sign,to_messages_DivBy_from_poi_to_this_person_sign,to_messages_DivBy_shared_receipt_with_poi_sign,from_poi_to_this_person_DivBy_to_messages_sign,from_poi_to_this_person_DivBy_shared_receipt_with_poi_sign,shared_receipt_with_poi_DivBy_to_messages_sign,shared_receipt_with_poi_DivBy_from_poi_to_this_person_sign,from_messages_DivBy_from_this_person_to_poi_sign,from_this_person_to_poi_DivBy_from_messages_sign
ECHOLS JOHN B,True,True,True,True,True,True,True,True,NaN,NaN,...,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
KAMINSKI WINCENTY J,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
BOWEN JR RAYMOND M,True,True,True,True,True,NaN,True,True,True,True,...,NaN,True,True,True,True,True,True,True,True,True
PIPER GREGORY F,True,True,NaN,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
HAYES ROBERT E,NaN,NaN,NaN,NaN,NaN,NaN,True,True,True,True,...,NaN,True,True,True,True,True,True,True,NaN,False


### Scaling

In [15]:
### Just do min-max on floats, not bools (objects for now because np.nan)

float_feats_lst = fin_features + email_features + pay_feats_divby_lst \
            + stock_feats_divby_lst + email_feats_divby_lst + quant_flags_lst
bool_feats_lst = out_flags_lst + sign_flags_lst

scaler = MinMaxScaler()
train_floats = pd.DataFrame(\
                        data=scaler.fit_transform(X=X_train[float_feats_lst]),
                        columns=X_train[float_feats_lst].columns,
                        index=X_train[float_feats_lst].index)
X_train_scaled = pd.concat(objs=[train_floats, X_train[bool_feats_lst]],
                           axis=1)
X_train_scaled.info()
X_train_scaled.head()

test_floats = pd.DataFrame(\
                        data=scaler.transform(X=X_test[float_feats_lst]),
                        columns=X_test[float_feats_lst].columns,
                        index=X_test[float_feats_lst].index)
X_test_scaled = pd.concat(objs=[test_floats, X_test[bool_feats_lst]],
                          axis=1)
X_test_scaled.info()
X_test_scaled.head()

<class 'pandas.core.frame.DataFrame'>
Index: 86 entries, ECHOLS JOHN B to MURRAY JULIA H
Columns: 282 entries, salary to from_this_person_to_poi_DivBy_from_messages_sign
dtypes: float64(235), object(47)
memory usage: 190.1+ KB


,salary,bonus,long_term_incentive,expenses,total_payments,exercised_stock_options,restricted_stock,total_stock_value,to_messages,from_poi_to_this_person,...,total_stock_value_DivBy_exercised_stock_options_sign,total_stock_value_DivBy_restricted_stock_sign,to_messages_DivBy_from_poi_to_this_person_sign,to_messages_DivBy_shared_receipt_with_poi_sign,from_poi_to_this_person_DivBy_to_messages_sign,from_poi_to_this_person_DivBy_shared_receipt_with_poi_sign,shared_receipt_with_poi_DivBy_to_messages_sign,shared_receipt_with_poi_DivBy_from_poi_to_this_person_sign,from_messages_DivBy_from_this_person_to_poi_sign,from_this_person_to_poi_DivBy_from_messages_sign
ECHOLS JOHN B,0.102281,0.023508,1.000000,0.091733,0.303062,0.018630,0.042848,0.031888,NaN,NaN,...,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
KAMINSKI WINCENTY J,0.192009,0.059675,0.136531,0.363709,0.116274,0.026717,0.009365,0.030817,0.380976,0.134426,...,True,True,True,True,True,True,True,True,True,True
BOWEN JR RAYMOND M,0.195391,0.231465,0.488524,0.286230,0.300417,NaN,0.024356,0.007263,0.150800,0.459016,...,NaN,True,True,True,True,True,True,True,True,True
PIPER GREGORY F,0.116627,0.059675,NaN,0.186082,0.191991,0.027702,0.043092,0.027702,0.098886,0.200000,...,True,True,True,True,True,True,True,True,True,True
HAYES ROBERT E,NaN,NaN,NaN,NaN,NaN,NaN,0.012385,0.003989,0.037428,0.052459,...,NaN,True,True,True,True,True,True,True,NaN,False


<class 'pandas.core.frame.DataFrame'>
Index: 37 entries, WESTFAHL RICHARD K to PRENTICE JAMES
Columns: 282 entries, salary to from_this_person_to_poi_DivBy_from_messages_sign
dtypes: bool(1), float64(235), object(46)
memory usage: 81.6+ KB


,salary,bonus,long_term_incentive,expenses,total_payments,exercised_stock_options,restricted_stock,total_stock_value,to_messages,from_poi_to_this_person,...,total_stock_value_DivBy_exercised_stock_options_sign,total_stock_value_DivBy_restricted_stock_sign,to_messages_DivBy_from_poi_to_this_person_sign,to_messages_DivBy_shared_receipt_with_poi_sign,from_poi_to_this_person_DivBy_to_messages_sign,from_poi_to_this_person_DivBy_shared_receipt_with_poi_sign,shared_receipt_with_poi_DivBy_to_messages_sign,shared_receipt_with_poi_DivBy_from_poi_to_this_person_sign,from_messages_DivBy_from_this_person_to_poi_sign,from_this_person_to_poi_DivBy_from_messages_sign
WESTFAHL RICHARD K,-0.012229,NaN,0.100146,0.224708,0.078499,NaN,0.040163,0.011586,NaN,NaN,...,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SHELBY REX,0.130883,0.023508,NaN,0.097667,0.222968,0.051911,0.097772,0.080190,0.014067,0.042623,...,True,True,True,True,True,True,True,True,True,True
DEFFNER JOSEPH M,0.125352,0.095841,0.142958,0.179810,0.130448,-0.000372,0.011245,0.004243,0.055011,0.377049,...,True,True,True,True,True,True,True,True,True,True
WASAFF GEORGE,0.177413,0.046112,0.069756,NaN,0.110175,0.053338,0.040548,0.065966,0.028720,0.072131,...,True,True,True,True,True,True,True,True,True,True
HANNON KEVIN P,0.161272,0.258590,0.836132,0.146558,0.023417,0.179235,0.095850,0.206989,0.082726,0.104918,...,True,True,True,True,True,True,True,True,True,True


### Imputation

Try three different methods.

Univariate: 1) float:0, bool:mode, 2) float:median, bool:mode

Multivariate: 3) floats: iterative imputer, default bools:mode

Not going to add "missingness" columns (add_indicator) because I'm guessing it will bias the model toward nonPOI recall without adding any value, plus it would double my already sizable feature set.

In [16]:
### 1) Floats to 0 and flags to mode.
### Restore bools to bool (from object because np.nan)

imp0 = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)
train_df = pd.DataFrame(\
            data=imp0.fit_transform(X=X_train_scaled[float_feats_lst]),
            columns=X_train[float_feats_lst].columns,
            index=X_train.index)
test_df = pd.DataFrame(data=imp0.transform(X=X_test_scaled[float_feats_lst]),
                        columns=X_test[float_feats_lst].columns,
                        index=X_test.index)

imp_mod = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
train_bool_df = (pd.DataFrame(\
            data=imp_mod.fit_transform(X=X_train_scaled[bool_feats_lst]),
            columns=X_train[bool_feats_lst].columns,
            index=X_train.index)).astype(bool)
test_bool_df = pd.DataFrame(\
            data=imp_mod.transform(X=X_test_scaled[bool_feats_lst]),
            columns=X_test[bool_feats_lst].columns,
            index=X_test.index).astype(bool)
X_train_scaled_imp0 = pd.concat(objs=[train_df, train_bool_df], axis=1)
X_test_scaled_imp0 = pd.concat(objs=[test_df, test_bool_df], axis=1)

In [17]:
np.any(np.isnan(X_train_scaled_imp0))
np.all(np.isfinite(X_train_scaled_imp0))
X_train_scaled_imp0.info()

np.any(np.isnan(X_test_scaled_imp0))
np.all(np.isfinite(X_test_scaled_imp0))
X_test_scaled_imp0.info()

False

True

<class 'pandas.core.frame.DataFrame'>
Index: 86 entries, ECHOLS JOHN B to MURRAY JULIA H
Columns: 282 entries, salary to from_this_person_to_poi_DivBy_from_messages_sign
dtypes: bool(94), float64(188)
memory usage: 134.9+ KB


False

True

<class 'pandas.core.frame.DataFrame'>
Index: 37 entries, WESTFAHL RICHARD K to PRENTICE JAMES
Columns: 282 entries, salary to from_this_person_to_poi_DivBy_from_messages_sign
dtypes: bool(94), float64(188)
memory usage: 58.0+ KB


In [18]:
### 2) Floats to median and bools to mode.

imp_med = SimpleImputer(missing_values=np.nan, strategy='median')
train_df = pd.DataFrame(\
            data=imp_med.fit_transform(X=X_train_scaled[float_feats_lst]),
            columns=X_train[float_feats_lst].columns,
            index=X_train.index)
test_df = pd.DataFrame(\
            data=imp_med.transform(X=X_test_scaled[float_feats_lst]),
            columns=X_test[float_feats_lst].columns,
            index=X_test.index)

X_train_scaled_imp_med = pd.concat(objs=[train_df, train_bool_df], axis=1)
X_test_scaled_imp_med = pd.concat(objs=[test_df,  test_bool_df], axis=1)

np.any(np.isnan(X_train_scaled_imp_med))
np.all(np.isfinite(X_train_scaled_imp_med))
X_train_scaled_imp_med.info()

np.any(np.isnan(X_test_scaled_imp_med))
np.all(np.isfinite(X_test_scaled_imp_med))
X_test_scaled_imp_med.info()

False

True

<class 'pandas.core.frame.DataFrame'>
Index: 86 entries, ECHOLS JOHN B to MURRAY JULIA H
Columns: 282 entries, salary to from_this_person_to_poi_DivBy_from_messages_sign
dtypes: bool(94), float64(188)
memory usage: 134.9+ KB


False

True

<class 'pandas.core.frame.DataFrame'>
Index: 37 entries, WESTFAHL RICHARD K to PRENTICE JAMES
Columns: 282 entries, salary to from_this_person_to_poi_DivBy_from_messages_sign
dtypes: bool(94), float64(188)
memory usage: 58.0+ KB


In [19]:
### 3) floats: iterative imputer, bools:mode

### Univar impute bools first to restore to bool and avoid imputing with floats.
X_train_scaled_imp_mv = pd.concat(objs=[X_train_scaled[float_feats_lst],
                                        train_bool_df], axis=1)
X_test_scaled_imp_mv = pd.concat(objs=[X_test_scaled[float_feats_lst],
                                       test_bool_df], axis=1)
X_train_scaled_imp_mv.info()
X_test_scaled_imp_mv.info()

### From https://scikit-learn.org/stable/modules/impute.html
### Note This estimator is still experimental for now: default parameters or
### details of behaviour might change without any deprecation cycle. Resolving
### the following issues would help stabilize IterativeImputer: convergence
### criteria (#14338), default estimators (#13286), and use of random state
###(#15611). To use it, you need to explicitly import enable_iterative_imputer.

imp_mv = IterativeImputer(missing_values=np.nan,#estimator=BayesianRidge(),
    sample_posterior=False, max_iter=10, tol=1e-3, n_nearest_features=None,
    initial_strategy='median', imputation_order='ascending',
    skip_complete=True, min_value=-np.inf, max_value=np.inf, random_state=42,
    add_indicator=False)

X_train_scaled_imp_mv = pd.DataFrame(\
            data=imp_mv.fit_transform(X=X_train_scaled_imp_mv),
            columns=X_train.columns, index=X_train.index)
X_test_scaled_imp_mv = pd.DataFrame(\
            data=imp_mv.transform(X=X_test_scaled_imp_mv),
            columns=X_test.columns, index=X_test.index)

X_train_scaled_imp_mv[bool_feats_lst] = \
    X_train_scaled_imp_mv[bool_feats_lst].astype(bool)
X_test_scaled_imp_mv[bool_feats_lst] = \
    X_test_scaled_imp_mv[bool_feats_lst].astype(bool)


np.any(np.isnan(X_train_scaled_imp_mv))
np.all(np.isfinite(X_train_scaled_imp_mv))
X_train_scaled_imp_mv.info()

np.any(np.isnan(X_test_scaled_imp_mv))
np.all(np.isfinite(X_test_scaled_imp_mv))
X_test_scaled_imp_mv.info()

<class 'pandas.core.frame.DataFrame'>
Index: 86 entries, ECHOLS JOHN B to MURRAY JULIA H
Columns: 282 entries, salary to from_this_person_to_poi_DivBy_from_messages_sign
dtypes: bool(94), float64(188)
memory usage: 134.9+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 37 entries, WESTFAHL RICHARD K to PRENTICE JAMES
Columns: 282 entries, salary to from_this_person_to_poi_DivBy_from_messages_sign
dtypes: bool(94), float64(188)
memory usage: 58.0+ KB


C:\Users\kaleb\anaconda3\envs\py3\lib\site-packages\sklearn\impute\_iterative.py:669: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn("[IterativeImputer] Early stopping criterion not"


False

True

<class 'pandas.core.frame.DataFrame'>
Index: 86 entries, ECHOLS JOHN B to MURRAY JULIA H
Columns: 282 entries, salary to from_this_person_to_poi_DivBy_from_messages_sign
dtypes: bool(94), float64(188)
memory usage: 134.9+ KB


False

True

<class 'pandas.core.frame.DataFrame'>
Index: 37 entries, WESTFAHL RICHARD K to PRENTICE JAMES
Columns: 282 entries, salary to from_this_person_to_poi_DivBy_from_messages_sign
dtypes: bool(94), float64(188)
memory usage: 58.0+ KB


### sklearn predictions as features

1) Linear regression of POI as float. An ML prediction as a boosting feature.

2) Kmeans cluster. I'm not going to test this idea on this project, but my hunch is that using an unsupervised cluster prediction as a boosting feature and using more n_clusters than there are target classes might aid POI recall by sort of dicing up potentially confounding classes (e.g. executive/nonexecutive), and by potentially creating more groups that are more biased toward POIs than the whole set and thus "boosting" the final algorithm with more quality information. I'll use five clusters. There may be a law of diminishing returns as variance may increase too much with too many clusters. I could write an algorithm to find which number of clusters gives the most clusters that have average POI values closest to 1 or 0, mostly 1. It would ne kind of a three-fold optimization: distance average is from 0 or 1, weighted for 1, by number of clusters. It could be cheaply expressed as average distance cluster averages are from 0 where cluster average is above 0.5 plus weight times average distance cluster averages are from 1 where cluster average is below 0.5, but this is well beyond my scope, and I have a lot to learn which may completely turn this thought on its head, and if not, I have future opportunities to test this idea.

In [20]:
### 1) Linear regression
reg_imp0 = LinearRegression().fit(X=X_train_scaled_imp0,
                                  y=y_train.astype(float))
X_train_scaled_imp0['lin_reg'] = reg_imp0.predict(X=X_train_scaled_imp0)
X_test_scaled_imp0['lin_reg'] = reg_imp0.predict(X=X_test_scaled_imp0)

reg_imp_med = LinearRegression().fit(X=X_train_scaled_imp_med,
                                  y=y_train.astype(float))
X_train_scaled_imp_med['lin_reg'] = \
                                reg_imp_med.predict(X=X_train_scaled_imp_med)
X_test_scaled_imp_med['lin_reg'] = reg_imp_med.predict(X=X_test_scaled_imp_med)

reg_imp_mv = LinearRegression().fit(X=X_train_scaled_imp_mv,
                                  y=y_train.astype(float))
X_train_scaled_imp_mv['lin_reg'] = reg_imp_mv.predict(X=X_train_scaled_imp_mv)
X_test_scaled_imp_mv['lin_reg'] = reg_imp_mv.predict(X=X_test_scaled_imp_mv)

pd.concat(objs=[X_train_scaled_imp0['lin_reg'], y_train], axis=1)
pd.concat(objs=[X_train_scaled_imp_med['lin_reg'], y_train], axis=1)
pd.concat(objs=[X_train_scaled_imp_mv['lin_reg'], y_train], axis=1)

,lin_reg,poi
ECHOLS JOHN B,6.383782e-16,0
KAMINSKI WINCENTY J,-9.159340e-16,0
BOWEN JR RAYMOND M,1.000000e+00,1
PIPER GREGORY F,7.285839e-16,0
HAYES ROBERT E,-3.330669e-16,0
...,...,...
HIRKO JOSEPH,1.000000e+00,1
GIBBS DANA R,1.776357e-15,0
SHARP VICTORIA T,-6.938894e-16,0
PIRO JIM,1.595946e-16,0


,lin_reg,poi
ECHOLS JOHN B,1.110223e-15,0
KAMINSKI WINCENTY J,1.887379e-15,0
BOWEN JR RAYMOND M,1.000000e+00,1
PIPER GREGORY F,5.551115e-16,0
HAYES ROBERT E,-2.220446e-16,0
...,...,...
HIRKO JOSEPH,1.000000e+00,1
GIBBS DANA R,-9.992007e-16,0
SHARP VICTORIA T,-1.887379e-15,0
PIRO JIM,-5.551115e-16,0


,lin_reg,poi
ECHOLS JOHN B,1.332268e-15,0
KAMINSKI WINCENTY J,-1.221245e-15,0
BOWEN JR RAYMOND M,1.000000e+00,1
PIPER GREGORY F,-4.440892e-16,0
HAYES ROBERT E,0.000000e+00,0
...,...,...
HIRKO JOSEPH,1.000000e+00,1
GIBBS DANA R,3.719247e-15,0
SHARP VICTORIA T,-1.776357e-15,0
PIRO JIM,2.775558e-15,0


In [21]:
### 2) Kmeans cluster
n_clusters = 5
clf_imp0 = KMeans(n_clusters=n_clusters, random_state=42)
X_train_scaled_imp0['k_means'] = clf_imp0.fit_predict(X=X_train_scaled_imp0)
X_test_scaled_imp0['k_means'] = clf_imp0.predict(X=X_test_scaled_imp0)

clf_imp_med = KMeans(n_clusters=n_clusters, random_state=42)
X_train_scaled_imp_med['k_means'] = \
                            clf_imp_med.fit_predict(X=X_train_scaled_imp_med)
X_test_scaled_imp_med['k_means'] = clf_imp_med.predict(X=X_test_scaled_imp_med)

clf_imp_mv = KMeans(n_clusters=n_clusters, random_state=42)
X_train_scaled_imp_mv['k_means'] = \
                            clf_imp_mv.fit_predict(X=X_train_scaled_imp_mv)
X_test_scaled_imp_mv['k_means'] = clf_imp_mv.predict(X=X_test_scaled_imp_mv)

pd.concat(objs=[X_train_scaled_imp0['k_means'], y_train], axis=1)
pd.concat(objs=[X_train_scaled_imp_med['k_means'], y_train], axis=1)
pd.concat(objs=[X_train_scaled_imp_mv['k_means'], y_train], axis=1)

,k_means,poi
ECHOLS JOHN B,2,0
KAMINSKI WINCENTY J,0,0
BOWEN JR RAYMOND M,4,1
PIPER GREGORY F,0,0
HAYES ROBERT E,1,0
...,...,...
HIRKO JOSEPH,1,1
GIBBS DANA R,1,0
SHARP VICTORIA T,0,0
PIRO JIM,1,0


,k_means,poi
ECHOLS JOHN B,1,0
KAMINSKI WINCENTY J,0,0
BOWEN JR RAYMOND M,2,1
PIPER GREGORY F,3,0
HAYES ROBERT E,4,0
...,...,...
HIRKO JOSEPH,3,1
GIBBS DANA R,4,0
SHARP VICTORIA T,0,0
PIRO JIM,4,0


,k_means,poi
ECHOLS JOHN B,2,0
KAMINSKI WINCENTY J,2,0
BOWEN JR RAYMOND M,0,1
PIPER GREGORY F,2,0
HAYES ROBERT E,3,0
...,...,...
HIRKO JOSEPH,4,1
GIBBS DANA R,1,0
SHARP VICTORIA T,4,0
PIRO JIM,1,0


In [22]:
with open('X_train.pkl', 'wb') as file:
    pickle.dump(obj=X_train, file=file)
with open('y_train.pkl', 'wb') as file:
    pickle.dump(obj=y_train, file=file)
with open('X_test.pkl', 'wb') as file:
    pickle.dump(obj=X_test, file=file)
with open('y_test.pkl', 'wb') as file:
    pickle.dump(obj=y_test, file=file)
with open('X_train_scaled_imp0.pkl', 'wb') as file:
    pickle.dump(obj=X_train_scaled_imp0, file=file)
with open('X_test_scaled_imp0.pkl', 'wb') as file:
    pickle.dump(obj=X_test_scaled_imp0, file=file)
with open('X_train_scaled_imp_med.pkl', 'wb') as file:
    pickle.dump(obj=X_train_scaled_imp_med, file=file)
with open('X_test_scaled_imp_med.pkl', 'wb') as file:
    pickle.dump(obj=X_test_scaled_imp_med, file=file)
with open('X_train_scaled_imp_mv.pkl', 'wb') as file:
    pickle.dump(obj=X_train_scaled_imp_mv, file=file)
with open('X_test_scaled_imp_mv.pkl', 'wb') as file:
    pickle.dump(obj=X_test_scaled_imp_mv, file=file)
    
with open('fin_features.pkl', 'wb') as file:
    pickle.dump(obj=fin_features, file=file)
with open('pay_features.pkl', 'wb') as file:
    pickle.dump(obj=pay_features, file=file)
with open('stock_features.pkl', 'wb') as file:
    pickle.dump(obj=stock_features, file=file)
with open('email_features.pkl', 'wb') as file:
    pickle.dump(obj=email_features, file=file)
with open('pay_feats_divby_lst.pkl', 'wb') as file:
    pickle.dump(obj=pay_feats_divby_lst, file=file)
with open('stock_feats_divby_lst.pkl', 'wb') as file:
    pickle.dump(obj=stock_feats_divby_lst, file=file)
with open('email_feats_divby_lst.pkl', 'wb') as file:
    pickle.dump(obj=email_feats_divby_lst, file=file)
with open('quant_flags_lst.pkl', 'wb') as file:
    pickle.dump(obj=quant_flags_lst, file=file)
with open('sign_flags_lst.pkl', 'wb') as file:
    pickle.dump(obj=sign_flags_lst, file=file)
features_list = ['poi'] + fin_features + email_features + pay_feats_divby_lst \
    + stock_feats_divby_lst + email_feats_divby_lst + quant_flags_lst \
    + sign_flags_lst
with open('features_list.pkl', 'wb') as file:
    pickle.dump(obj=features_list, file=file)

In [24]:
### Store to my_dataset for easy export below.
my_dataset = data_dict

### Extract features and labels from dataset for local testing
data = featureFormat(my_dataset, features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)

NameError: name 'data_dict' is not defined

In [ ]:
### Task 4: Try a varity of classifiers
### Please name your classifier clf for easy export below.
### Note that if you want to do PCA or other multi-stage operations,
### you'll need to use Pipelines. For more info:
### http://scikit-learn.org/stable/modules/pipeline.html

# Provided to give you a starting point. Try a variety of classifiers.
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()

In [ ]:
### Task 5: Tune your classifier to achieve better than .3 precision and recall 
### using our testing script. Check the tester.py script in the final project
### folder for details on the evaluation method, especially the test_classifier
### function. Because of the small size of the dataset, the script uses
### stratified shuffle split cross validation. For more info: 
### http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.StratifiedShuffleSplit.html

# Example starting point. Try investigating other evaluation techniques!
from sklearn.cross_validation import train_test_split
features_train, features_test, labels_train, labels_test = \
    train_test_split(features, labels, test_size=0.3, random_state=42)

In [ ]:
### Task 6: Dump your classifier, dataset, and features_list so anyone can
### check your results. You do not need to change anything below, but make sure
### that the version of poi_id.py that you submit can be run on its own and
### generates the necessary .pkl files for validating your results.

dump_classifier_and_data(clf, my_dataset, features_list)